In [1]:
import tweepy
import time
import networkx as nx
import pandas as pd
import json
import numpy as np
import requests
from bs4 import BeautifulSoup
from twitterscraper import query_tweets
from datetime import datetime
from networkx.readwrite import json_graph
%matplotlib inline



In [2]:
consumer_key = "2e6dNamYAUwvPcLJ5YKn9ow6R"
consumer_secret = "VFujaQNDkF6d7YKrTttJzMhCfNpFcYCnMyDSvjGssEvq4WYvnm"
access_token = "939149568519815169-c6Lti1dhyk5Z8wmwymsrpvw1BtxVE0Z"
access_token_secret = "IaQwQlN3yrkoZsB5nzi8Ha2AO1TseVdLcPJgltP6ljZiI"   

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [83]:
res = api.get_user('nasw')

In [84]:
G = nx.DiGraph()
G.add_node(res.id, attr_dict = {'status_count': res.statuses_count, 'screen_name': res.screen_name})

In [85]:
G.nodes(data=True)

NodeDataView({17869267: {'attr_dict': {'status_count': 7534, 'screen_name': 'nasw'}}})

In [98]:
res = api.get_user('nasw')

In [105]:
G = nx.DiGraph()
G.add_node(res.id, attr_dict = {'status_count': res.statuses_count, 'screen_name': res.screen_name})

In [106]:
handles = ["Dr_Faustino", "epflcswccm", "HarmonicFamily", "SocialWorkItOut", "cheryl_aguilar"]
with open("followers2.txt", "w") as file:
    for handle in handles:
        user = api.get_user(handle)
        G.add_node(user.id, attr_dict = {'status_count': user.statuses_count, 'screen_name': user.screen_name})
        file.write(str(user.id) + '\t' + str(res.id) + '\n')

In [112]:
def get_network(filename, G2=None):
    G = nx.read_edgelist(filename, create_using=nx.DiGraph(), nodetype=int)
    if G2 != None:
        G = nx.compose(G, G2)
    return G

In [117]:
G = get_network('followers2.txt', G)

In [121]:
G.nodes(data=True)

NodeDataView({241400800: {'attr_dict': {'status_count': 1320, 'screen_name': 'SocialWorkItOut'}}, 17869267: {'attr_dict': {'status_count': 7534, 'screen_name': 'nasw'}}, 26420085: {'attr_dict': {'status_count': 8863, 'screen_name': 'cheryl_aguilar'}}, 47505197: {'attr_dict': {'status_count': 2556, 'screen_name': 'Dr_Faustino'}}, 159073678: {'attr_dict': {'status_count': 16397, 'screen_name': 'epflcswccm'}}, 193075453: {'attr_dict': {'status_count': 3861, 'screen_name': 'HarmonicFamily'}}})

In [43]:
def fetchGraph(screen_name, max_tweets,max_retweets):
    res = api.get_user(screen_name)
    G = nx.DiGraph()
    G.add_node(res.id, attr_dict = {'status_count': res.statuses_count, 'screen_name': res.screen_name})
    for tweet in tweepy.Cursor(api.user_timeline, id=res.id).items(max_tweets):
        retweets = api.retweets(id=tweet.id, count=max_retweets)
        for retweet in retweets:
            retweet_id = retweet.user.id
            G.add_node(retweet_id, attr_dict = {'status_count': retweet.user.statuses_count, 
                                                         'screen_name': retweet.user.screen_name})
            if G.has_edge(retweet_id, res.id):
                G.add_edge(retweet_id, res.id, weight=G[retweet_id][res.id]['weight'] + 1)
            else:
                G.add_edge(retweet_id, res.id, weight=1)
    print(len(G.nodes()))
    return G
            
            

In [47]:
G = fetchGraph('nasw', 100, 50)

Rate limit reached. Sleeping for: 636
55


In [48]:
G.edges(data=True)

OutEdgeDataView([(770412243095658496, 17869267, {'weight': 100}), (702288726546735104, 17869267, {'weight': 100}), (931899603858927618, 17869267, {'weight': 100}), (898301089820364804, 17869267, {'weight': 100}), (743469564625752064, 17869267, {'weight': 100}), (754055663236419585, 17869267, {'weight': 100}), (263474444, 17869267, {'weight': 100}), (975865357, 17869267, {'weight': 100}), (2250262670, 17869267, {'weight': 100}), (3094761492, 17869267, {'weight': 100}), (2758789909, 17869267, {'weight': 88}), (2946901143, 17869267, {'weight': 100}), (216039108, 17869267, {'weight': 100}), (246364828, 17869267, {'weight': 35}), (60331933, 17869267, {'weight': 65}), (899276532950204417, 17869267, {'weight': 65}), (767952194108653568, 17869267, {'weight': 100}), (1468131488, 17869267, {'weight': 35}), (1401470755, 17869267, {'weight': 100}), (744735996391002113, 17869267, {'weight': 65}), (2211542150, 17869267, {'weight': 100}), (614056227, 17869267, {'weight': 100}), (852462506, 17869267, 

In [15]:
def saveGraph(G, filename):
    data = json_graph.node_link_data(G)
    json_data = json.dumps(data)
    with open(filename, 'w') as file:
        file.write(json_data)

In [50]:
saveGraph(G, 'retweetfriendGraph.json')

In [3]:
res = api.get_user('nasw')
for tweet in tweepy.Cursor(api.user_timeline, id=res.id).items(1):
    print(tweet)

Status(id=957621880877408256, retweet_count=10, geo=None, coordinates=None, in_reply_to_screen_name=None, created_at=datetime.datetime(2018, 1, 28, 14, 30, 22), in_reply_to_user_id_str=None, _json={'retweet_count': 10, 'geo': None, 'coordinates': None, 'in_reply_to_screen_name': None, 'created_at': 'Sun Jan 28 14:30:22 +0000 2018', 'in_reply_to_user_id_str': None, 'in_reply_to_status_id_str': None, 'extended_entities': {'media': [{'id': 957621878167953413, 'expanded_url': 'https://twitter.com/nasw/status/957621880877408256/photo/1', 'url': 'https://t.co/dS8l83ANA6', 'media_url_https': 'https://pbs.twimg.com/media/DUooBkVX4AUw-tI.jpg', 'indices': [110, 133], 'display_url': 'pic.twitter.com/dS8l83ANA6', 'media_url': 'http://pbs.twimg.com/media/DUooBkVX4AUw-tI.jpg', 'type': 'photo', 'id_str': '957621878167953413', 'sizes': {'small': {'h': 339, 'w': 671, 'resize': 'fit'}, 'large': {'h': 339, 'w': 671, 'resize': 'fit'}, 'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}, 'medium': {'h': 339, '

In [9]:
tweet.created_at.date()

datetime.date(2018, 1, 28)

In [11]:
G = nx.DiGraph()
G.add_nodes_from([1, 2])

In [23]:
G.add_edge(1, 2, date=tweet.created_at.date().strftime('%m/%d/%Y'))

In [24]:
G.edges(data=True)

OutEdgeDataView([(1, 2, {'date': '01/28/2018'})])

In [25]:
saveGraph(G, 'sampleGraph.json')
# stringTime = tweet.created_at.date().strftime('%m/%d/%Y')

In [22]:
datetime.strptime(stringTime, '%m/%d/%Y').date()

datetime.date(2018, 1, 28)

In [27]:
list_tweets = query_tweets("@nasw", 1000)

Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Process ForkPoolWorker-6:
Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-19:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Process ForkPoolWorker-5:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", lin

  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/

In [42]:
final_tweets = []
for tweet in list_tweets:
    if tweet.user == 'nasw':
        final_tweets.append(tweet.text)
len(final_tweets)

135

In [3]:
def fetchMentions(screen_name, max_tweets=1000):
    list_tweets = query_tweets(screen_name, max_tweets)
    dates = [tweet.timestamp.date().strftime('%d/%m/%Y') for tweet in list_tweets if tweet.user != screen_name]
    unique_dates = list(set(dates))
    unique_dates = [datetime.strptime(unique_date, '%d/%m/%Y').date() for unique_date in unique_dates]
    unique_dates.sort()
    unique_dates = [date.strftime('%d/%m/%Y') for date in unique_dates]
    mention_order = []
    for unique_date in unique_dates:
        mention_order.append(dates.count(unique_date))
    return mention_order

In [4]:
result = fetchMentions("nasw", 1000)

In [13]:
result

[19,
 38,
 27,
 31,
 53,
 55,
 11,
 14,
 31,
 9,
 22,
 29,
 17,
 25,
 12,
 3,
 11,
 30,
 14,
 28,
 28,
 24,
 33,
 15,
 41,
 2,
 22,
 34,
 25,
 31,
 37,
 22,
 6,
 45,
 3,
 55,
 15,
 36,
 60,
 14,
 46,
 1,
 20,
 34]

In [34]:
def getIncreasingSplit(result):
    series = []
    maxEle = 0
    initIndex = 0
    for index, element in enumerate(result):
        if maxEle >= element or element == 0 or index == (len(result) - 1):
            series.append(result[initIndex:index])
            initIndex = index
        maxEle = element
    return series

def getDrRank(result):
    result = getIncreasingSplit(result)
    result = [elements for elements in result if len(elements) > 2]
    ranks = []
    for elements in result:
        initIndex, endIndex = 0, -1
        if elements[initIndex] == 1:
            initIndex += 1
        ranks.append(elements[endIndex]/elements[initIndex])
    return max(ranks)
            

In [35]:
getDrRank(result)

17.0